# Sequentially Chaining the Agent Calls

Now that you have two activated ACP servers, you will create a sequential workflow by sequentially chaining the two agents as follows:
- the Hospital agent that handles health-based questions for patients will execute first;
- the Insurer agent that handles questions related to insurance coverage will execute secondly; 
- since these two agents are hosted on two separate servers, you will need two clients: each client will interact with one server.
  - the first client will send the user's query to the hospital server and receives the response from the server;
  - you will pass the response from the first client to the second client which will combine in its query and send it to the second server 

<img src="seq.png" width="650">

## Run the Insurer ACP Server

First make sure the Insurer server is still running:
- Open the terminal by running the cell below. Run the server by typing
  - `uv run crew_agent_server.py`

In [1]:
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=600)

## 7.2. Run the Hospital ACP Server

Also make sure the Hospital server is still running:
- Open the terminal by running the cell below.
- then you can run the server again by typing:
  - `uv run smolagents_server.py`

In [2]:
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/2", width=800, height=600)

## 7.3. Chain the LLM Calls 

You will now create two ACP clients, each will interact with one agent so that hospital agent will execute first and the insurer agent will execute secondly.

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from acp_sdk.client import Client
import asyncio
from colorama import Fore 

async def run_hospital_workflow() -> None:
    async with Client(base_url="http://localhost:8001") as insurer, Client(base_url="http://localhost:8000") as hospital:
        run1 = await hospital.run_sync(
            agent="health_agent", input="Do I need rehabilitation after a shoulder reconstruction?"
        )
        content = run1.output[0].parts[0].content
        print(Fore.LIGHTMAGENTA_EX+ content + Fore.RESET)

        run2 = await insurer.run_sync(
            agent="policy_agent", input=f"Context: {content} What is the waiting period for rehabilitation?"
        )
        print(Fore.YELLOW + run2.output[0].parts[0].content + Fore.RESET)

In [5]:

asyncio.run(run_hospital_workflow())

ConnectError: All connection attempts failed